#### Setup & Quote Extraction

In this phase, we move from Bars (OHLCV) to Quotes (Bid/Ask). The "Spread" is the most fundamental BI metric for liquidity.

- fetch the latest "Top of Book" quotes. This shows us the immediate gap between buyers and sellers.

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoQuoteRequest 
from datetime import datetime, timedelta, timezone # Added timezone

# Load credentials
load_dotenv()
client = CryptoHistoricalDataClient(api_key=os.getenv('ALPACA_PAPER_KEY'), 
                                    secret_key=os.getenv('ALPACA_PAPER_SECRET'))

# Symbols to profile
symbols = ["BTC/USD", "SOL/USD", "PEPE/USD"]

# Use timezone.utc to ensure the API understands the request correctly
now_utc = datetime.now(timezone.utc)
start_utc = now_utc - timedelta(minutes=60)

# Create the request with explicit UTC times
request_params = CryptoQuoteRequest(
    symbol_or_symbols=symbols,
    start=start_utc,
    end=now_utc,      # Explicitly setting the end time helps avoid default errors
    limit=500
)

# Fetch quotes
quotes = client.get_crypto_quotes(request_params)
quotes_df = quotes.df.reset_index()

print(f"Captured {len(quotes_df)} real-time market quotes using UTC.")
quotes_df.head()

Captured 500 real-time market quotes using UTC.


,symbol,timestamp,bid_price,bid_size,ask_price,ask_size
0,BTC/USD,2026-01-29 06:17:21.884722+00:00,88120.90,1.27440,88230.381,1.2811
1,BTC/USD,2026-01-29 06:17:21.884876+00:00,88120.90,1.27440,88238.784,2.5514
2,BTC/USD,2026-01-29 06:17:21.884884+00:00,88120.90,1.27440,88225.460,1.2768
3,BTC/USD,2026-01-29 06:17:21.884887+00:00,88108.91,1.27630,88225.460,1.2768
4,BTC/USD,2026-01-29 06:17:21.960418+00:00,88096.40,2.56803,88225.460,1.2768


#### Calculating the "Liquidity Spread" KPI

In [3]:
# 1. Calculate Mid-Price
quotes_df['mid_price'] = (quotes_df['ask_price'] + quotes_df['bid_price']) / 2

# 2. Calculate Quoted Spread (Absolute $)
quotes_df['abs_spread'] = quotes_df['ask_price'] - quotes_df['bid_price']

# 3. Calculate Relative Spread (Percentage) - Crucial for comparing BTC ($100k) to PEPE ($0.00001)
# Formula: (Ask - Bid) / Mid_Price
quotes_df['rel_spread_bps'] = (quotes_df['abs_spread'] / quotes_df['mid_price']) * 10000 # In Basis Points

# Group by symbol to see who has the "Thickest" liquidity
liquidity_summary = quotes_df.groupby('symbol')['rel_spread_bps'].agg(['mean', 'max', 'std']).round(2)
print("Liquidity Profile (Spread in Basis Points - Lower is Better):")
print(liquidity_summary)

Liquidity Profile (Spread in Basis Points - Lower is Better):
          mean    max   std
symbol                     
BTC/USD  11.62  17.85  2.78


#### Data Export for Scenario Modeling

In [4]:
quotes_df.to_csv('market_depth_quotes.csv', index=False)
liquidity_summary.to_csv('liquidity_metrics.csv')

print("Liquidity profiles generated.")

Liquidity profiles generated.
